In [10]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
from collections import ChainMap
import os

### CLEAN UP OF MAIN ABD DATAFRAME

In [2]:
#Load ABD main data
ABD_main = pd.read_csv('ABD_ballToball.csv')

In [3]:
#Dropping unnecessary column
ABD_main.drop(columns = 'Unnamed: 0', inplace = True)

In [4]:
#Appropriate Col name
ABD_main.rename(columns = {'Year' : 'Date', 'Non_Striker' : 'Non Striker'}, inplace = True)

In [5]:
#Convert date string to timestamp
ABD_main['Date'] = ABD_main['Date'].apply(pd.to_datetime)

In [6]:
ABD_main.head()

,Over,Bowler,Non Striker,Runs,Extras,ExtraType,Status,BattingOrder,WicketsInHand,Opposition,Venue,Date
0,1.5,Sandeep Sharma,Vishnu Vinod,4,0,legal,Not Out,3,7,Kings XI Punjab,Indore,2017-04-10
1,1.6,Sandeep Sharma,Vishnu Vinod,0,0,legal,Not Out,3,7,Kings XI Punjab,Indore,2017-04-10
2,2.4,MM Sharma,Vishnu Vinod,0,0,legal,Not Out,3,7,Kings XI Punjab,Indore,2017-04-10
3,2.5,MM Sharma,Vishnu Vinod,0,0,legal,Not Out,3,7,Kings XI Punjab,Indore,2017-04-10
4,2.6,MM Sharma,Vishnu Vinod,6,0,legal,Not Out,3,7,Kings XI Punjab,Indore,2017-04-10


In [7]:
#Changing team names
old_names = ['Deccan Chargers', 'Kings XI Punjab', 'Rising Pune Supergiant', 'Delhi Daredevils']
new_names = ['Sunrisers Hyderabad', 'Punjab Kings', 'Rising Pune Supergiants', 'Delhi Capitals']
for old, new in zip(old_names, new_names):
    ABD_main.loc[ABD_main['Opposition'] == old, 'Opposition'] = new

In [8]:
ABD_main['Opposition'].unique()

array(['Punjab Kings', 'Mumbai Indians', 'Rising Pune Supergiants',
       'Kolkata Knight Riders', 'Gujarat Lions', 'Rajasthan Royals',
       'Delhi Capitals', 'Chennai Super Kings', 'Sunrisers Hyderabad',
       'Kochi Tuskers Kerala', 'Pune Warriors'], dtype=object)

In [13]:
import sys
sys.path.append(os.getcwd() + '\\DataAfterScraping\\')

In [15]:
#Load Bowler type
#This was due to having three files and the recapthca problem
#JOB: FIX recapthca
path = os.getcwd() + '\\DataAfterScraping\\'
with open(path + 'bowler_type_1.json') as f:
    bowler_type1 = json.load(f)
with open(path + 'bowler_type_2.json') as f:
    bowler_type2 = json.load(f)
with open(path + 'bowler_type_3.json') as f:
    bowler_type3 = json.load(f)
bowler_type = bowler_type1 + bowler_type2 + bowler_type3
#convert list of dicts into main dict
bowler_type_dict = dict(ChainMap(*bowler_type))

In [17]:
#Create a new column for bowler ype using map from bowler
#https://stackoverflow.com/questions/24216425/adding-a-new-pandas-column-with-mapped-value-from-a-dictionary
ABD_main['Bowler Type'] = ABD_main['Bowler'].map(bowler_type_dict)

In [18]:
#convert Overs numeric values into categorical
#https://stackoverflow.com/questions/49382207/how-to-map-numeric-data-into-categories-bins-in-pandas-dataframe

bins = [0, 6, 10, 16, 20]
names = ['Power Play', 'Middle Overs (6-10)', 'Middle Overs (10-16)', 'Death Overs']
ABD_main['Over category'] = pd.cut(ABD_main['Over'], bins, labels=names)

In [19]:
ABD_main.head()

,Over,Bowler,Non Striker,Runs,Extras,ExtraType,Status,BattingOrder,WicketsInHand,Opposition,Venue,Date,Bowler Type,Over category
0,1.5,Sandeep Sharma,Vishnu Vinod,4,0,legal,Not Out,3,7,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play
1,1.6,Sandeep Sharma,Vishnu Vinod,0,0,legal,Not Out,3,7,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play
2,2.4,MM Sharma,Vishnu Vinod,0,0,legal,Not Out,3,7,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play
3,2.5,MM Sharma,Vishnu Vinod,0,0,legal,Not Out,3,7,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play
4,2.6,MM Sharma,Vishnu Vinod,6,0,legal,Not Out,3,7,Punjab Kings,Indore,2017-04-10,Right-arm fast-medium,Power Play


In [20]:
new_cols = ['Date', 'Over', 'Over category', 'BattingOrder', 'WicketsInHand', 'Bowler', 'Bowler Type', 'Runs', 'Extras', 'ExtraType', 'Status',  \
            'Non Striker', 'Opposition', 'Venue']
ABD_main = ABD_main[new_cols]

In [21]:
#Manual entry of missing columns for bowler types
#Bowling types of Raiphi gomez, kamran khan, Shalb srivastava, AN ahmed
missing_types = ['Right-arm medium', 'Left-arm fast-medium', 'Left-arm fast-medium', 'Right-arm fast-medium']
for bowler_missing_type, missing_type in \
                    zip(ABD_main.loc[ABD_main['Bowler Type'] == ' ', 'Bowler'].unique(), missing_types):
    ABD_main['Bowler Type'][ABD_main['Bowler'] == bowler_missing_type] = missing_type
#JOB understand the warning   

C:\Users\Owner\Anaconda3\envs\NeuralNets\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
#ABD_main.drop(columns = 'Over', inplace = True)

In [23]:
ABD_main.head(50)

,Date,Over,Over category,BattingOrder,WicketsInHand,Bowler,Bowler Type,Runs,Extras,ExtraType,Status,Non Striker,Opposition,Venue
0,2017-04-10,1.5,Power Play,3,7,Sandeep Sharma,Right-arm fast-medium,4,0,legal,Not Out,Vishnu Vinod,Punjab Kings,Indore
1,2017-04-10,1.6,Power Play,3,7,Sandeep Sharma,Right-arm fast-medium,0,0,legal,Not Out,Vishnu Vinod,Punjab Kings,Indore
2,2017-04-10,2.4,Power Play,3,7,MM Sharma,Right-arm fast-medium,0,0,legal,Not Out,Vishnu Vinod,Punjab Kings,Indore
3,2017-04-10,2.5,Power Play,3,7,MM Sharma,Right-arm fast-medium,0,0,legal,Not Out,Vishnu Vinod,Punjab Kings,Indore
4,2017-04-10,2.6,Power Play,3,7,MM Sharma,Right-arm fast-medium,6,0,legal,Not Out,Vishnu Vinod,Punjab Kings,Indore
5,2017-04-10,3.4,Power Play,3,6,Sandeep Sharma,Right-arm fast-medium,1,0,legal,Not Out,KM Jadhav,Punjab Kings,Indore
6,2017-04-10,4.1,Power Play,3,6,VR Aaron,Right-arm fast,0,0,legal,Not Out,KM Jadhav,Punjab Kings,Indore
7,2017-04-10,4.2,Power Play,3,6,VR Aaron,Right-arm fast,0,0,legal,Not Out,KM Jadhav,Punjab Kings,Indore
8,2017-04-10,4.3,Power Play,3,6,VR Aaron,Right-arm fast,1,0,legal,Not Out,KM Jadhav,Punjab Kings,Indore
9,2017-04-10,4.5,Power Play,3,6,VR Aaron,Right-arm fast,1,0,legal,Not Out,KM Jadhav,Punjab Kings,Indore


In [24]:
#Goal is prediction of a run based on overs - powerplay, middle overs, death overs (convert overs to categorical)
#                                      bowler_type - available
#                                      previous ball was a no ball? can be obtained easily - free hit means boundary attempt
#                                      bowler_economy previos matches how mnay previous years should I take? featutre problem
#                                      bowler_srike rate - same queston as before
#                                      wckets talen by owler same qestion as previous
#                                      venue - indicator of dimensions
#                                      non striker quality

In [25]:
with open(path + 'bowler_stats.json') as f:
    bowler_stats = json.load(f)

In [26]:
bowler_stats_dict = dict(ChainMap(*bowler_stats))

### Creating a data set with last three years + until that match bowler stats and non striker stats

In [27]:
#bowler_stats[0]['Sandeep Sharma'] #- List of years in decreasing order
#bowler_stats[0]['Sandeep Sharma'][0] - Year, Balls, Runs, Wickets, Best figures, Average, Economy, SR etc as keys
#Goal is to create a list (column) that checks for bolwer and the year first and gets prev three years data

ABD_main_V2 = ABD_main.copy()
ABD_main_V2.sort_values(by = 'Date', inplace = True)

In [28]:
#ABD_main_V2.index[0].year
def f(x,y, sub, label):
    #we are accessing row datetime and bowler name
    #scanning through all rows
  
    try:
        #bowler_stats_dict[y] # - has all years of a bowler
        bowler_year_dict = dict(ChainMap(*bowler_stats_dict[x]))
        
    except:
        
        #When there is no stat
        return np.nan
    
    
    #bowler_year_list
    all_years_list = list(bowler_year_dict.keys())
    
    #Pick the key and access the stats
    try:
        
        stat_for_that_year = float(bowler_year_dict[str(y.year- sub)][label])
        
    except:
        
        #when stat was accidently taken from the wrong website
        return np.nan

    return  stat_for_that_year

#https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe 
#ABD_main_V2['Econ_this_year'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 0, 'Economy'), axis = 1)
ABD_main_V2['Economy_Y - 1'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 1, 'Economy'), axis = 1)
ABD_main_V2['Economy_Y - 2'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 2, 'Economy'), axis = 1)
ABD_main_V2['Economy_Y - 3'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 3, 'Economy'), axis = 1)


#Wickets taken
ABD_main_V2['Wickets_Y - 1'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 1, 'Wickets'), axis = 1)
ABD_main_V2['Wickets_Y - 2'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 2, 'Wickets'), axis = 1)
ABD_main_V2['Wickets_Y - 3'] = ABD_main_V2.apply(lambda x : f(x.Bowler, x.Date, 3, 'Wickets'), axis = 1)

In [29]:
ABD_main_V2.dropna(inplace= True)

In [30]:
ABD_main_V2['3Yr_Econ_Avg'] = round((ABD_main_V2['Economy_Y - 1'] + ABD_main_V2['Economy_Y - 2']\
                                       + ABD_main_V2['Economy_Y - 3']) / 3 , 3)

ABD_main_V2['3Yr_Wickets_Total'] = ABD_main_V2['Wickets_Y - 1'] + ABD_main_V2['Wickets_Y - 2'] \
                                        + ABD_main_V2['Wickets_Y - 3']


In [31]:
ABD_main_V2.drop(columns = ['Economy_Y - 1', 'Economy_Y - 2', 'Economy_Y - 3', \
                           'Wickets_Y - 1', 'Wickets_Y - 2', 'Wickets_Y - 3'], inplace = True )

In [32]:
ABD_main_V2.head()

,Date,Over,Over category,BattingOrder,WicketsInHand,Bowler,Bowler Type,Runs,Extras,ExtraType,Status,Non Striker,Opposition,Venue,3Yr_Econ_Avg,3Yr_Wickets_Total
1194,2011-04-09,13.4,Middle Overs (10-16),4,5,R Vinay Kumar,Right-arm fast-medium,1,0,legal,Not Out,SS Tiwary,Kochi Tuskers Kerala,Kochi,8.287,30.0
1190,2011-04-09,12.2,Middle Overs (10-16),4,5,M Muralitharan,Right-arm offbreak,6,0,legal,Not Out,SS Tiwary,Kochi Tuskers Kerala,Kochi,6.343,40.0
1191,2011-04-09,12.3,Middle Overs (10-16),4,5,M Muralitharan,Right-arm offbreak,1,0,legal,Not Out,SS Tiwary,Kochi Tuskers Kerala,Kochi,6.343,40.0
1192,2011-04-09,13.2,Middle Overs (10-16),4,5,R Vinay Kumar,Right-arm fast-medium,0,0,legal,Not Out,SS Tiwary,Kochi Tuskers Kerala,Kochi,8.287,30.0
1193,2011-04-09,13.3,Middle Overs (10-16),4,5,R Vinay Kumar,Right-arm fast-medium,0,0,legal,Not Out,SS Tiwary,Kochi Tuskers Kerala,Kochi,8.287,30.0


In [33]:
ABD_main_V2.sort_values(by = ['Date', 'Over'], inplace= True)

In [34]:
##### IGNORE ###############

In [35]:
#ABD_main_V2['PreviousBallType'] = ABD_main_V2['Extra Type'].shift(1)

In [36]:
"""
test = ABD_main_V2[['Over', 'Extra Type']]
test['PrevBall'] = test['Over'].shift(1)
test.dropna(inplace = True)
test['DiffBall'] = test['Over'] - test['PrevBall']
def func(e, d):
    if float(d) == 0.1 and e != 'legal':
        return e
    else: return 'legal'
    
test['PrevBallType'] = test.apply(lambda x : func(x['Extra Type'], x['DiffBall']))
"""

"\ntest = ABD_main_V2[['Over', 'Extra Type']]\ntest['PrevBall'] = test['Over'].shift(1)\ntest.dropna(inplace = True)\ntest['DiffBall'] = test['Over'] - test['PrevBall']\ndef func(e, d):\n    if float(d) == 0.1 and e != 'legal':\n        return e\n    else: return 'legal'\n    \ntest['PrevBallType'] = test.apply(lambda x : func(x['Extra Type'], x['DiffBall']))\n"

In [37]:
###### END IGNORE ##########

In [38]:
#Rearrange columns
new_cols = ['Date', 'Over', 'Over category', 'BattingOrder', 'WicketsInHand',  'Non Striker', \
            'Bowler', 'Bowler Type', '3Yr_Econ_Avg', '3Yr_Wickets_Total', 'Status' \
                ,'Opposition', 'Venue', 'Runs' ]
ABD_main_V2 = ABD_main_V2[new_cols]

In [39]:
ABD_main_V2.head(200)

,Date,Over,Over category,BattingOrder,WicketsInHand,Non Striker,Bowler,Bowler Type,3Yr_Econ_Avg,3Yr_Wickets_Total,Status,Opposition,Venue,Runs
1166,2011-04-09,5.4,Power Play,4,6,MA Agarwal,R Vinay Kumar,Right-arm fast-medium,8.287,30.0,Not Out,Kochi Tuskers Kerala,Kochi,0
1167,2011-04-09,5.5,Power Play,4,6,MA Agarwal,R Vinay Kumar,Right-arm fast-medium,8.287,30.0,Not Out,Kochi Tuskers Kerala,Kochi,0
1168,2011-04-09,5.6,Power Play,4,6,MA Agarwal,R Vinay Kumar,Right-arm fast-medium,8.287,30.0,Not Out,Kochi Tuskers Kerala,Kochi,0
1169,2011-04-09,6.4,Middle Overs (6-10),4,6,MA Agarwal,M Muralitharan,Right-arm offbreak,6.343,40.0,Not Out,Kochi Tuskers Kerala,Kochi,2
1170,2011-04-09,6.5,Middle Overs (6-10),4,6,MA Agarwal,M Muralitharan,Right-arm offbreak,6.343,40.0,Not Out,Kochi Tuskers Kerala,Kochi,0
1171,2011-04-09,6.6,Middle Overs (6-10),4,6,MA Agarwal,M Muralitharan,Right-arm offbreak,6.343,40.0,Not Out,Kochi Tuskers Kerala,Kochi,0
1172,2011-04-09,7.4,Middle Overs (6-10),4,6,MA Agarwal,R Vinay Kumar,Right-arm fast-medium,8.287,30.0,Not Out,Kochi Tuskers Kerala,Kochi,1
1173,2011-04-09,8.1,Middle Overs (6-10),4,6,MA Agarwal,S Sreesanth,Right-arm fast-medium,9.007,28.0,Not Out,Kochi Tuskers Kerala,Kochi,1
1174,2011-04-09,8.3,Middle Overs (6-10),4,6,MA Agarwal,S Sreesanth,Right-arm fast-medium,9.007,28.0,Not Out,Kochi Tuskers Kerala,Kochi,1
1175,2011-04-09,8.5,Middle Overs (6-10),4,6,MA Agarwal,S Sreesanth,Right-arm fast-medium,9.007,28.0,Not Out,Kochi Tuskers Kerala,Kochi,1


In [40]:
#all_data['Order Day new'] = all_data['Order Day new'].dt.strftime('%Y-%m-%d')
ABD_main_V2['Date'] = ABD_main_V2['Date'].dt.strftime('%Y-%m-%d')

In [41]:
#Cumulatve runs scored by AB to any bowler
#Could be useful for exploratoty
#Without a bowler name could cum runs for a bowler (rep as type) represt AB laying safe or taking the bowler on
matchdate_bowler_group =  ABD_main_V2.groupby(['Date', 'Bowler', 'Over']).sum().groupby('Bowler').cumsum()
matchdate_bowler_group.drop(columns = ['3Yr_Econ_Avg', '3Yr_Wickets_Total'], inplace = True)
def func(d, b, o): 
    return matchdate_bowler_group.loc[d, b, o][0]
    
ABD_main_V2['CumulativeRunsForBowler'] = ABD_main_V2.apply(lambda x : func(x.Date, x.Bowler, x.Over), axis = 1)

In [42]:
#Quantifying the situation of the match
#Starting with batting position and wickets in hand
# We have batting position
ABD_main_V2['BattingOrder'].unique()

array([4, 5, 6, 3], dtype=int64)

In [44]:
ABD_main_V2.to_csv('Runs_ThreeYearAvgWickets.csv')